# 第二章第二节：组合逻辑

## 本节的目的
在这一节中，您将了解如何使用Chisel组件来实现组合逻辑。我们将演示三种基本的Chisel类型：`UInt `-无符号整数；`SInt`-有符号整数和`Bool`-  true或false。
注意所有Chisel变量都被声明为Scala的常量类型`val`。
永远不要将Scala的变量`var`用来构造硬件，因为硬件本身在定义之后就不应该再改变了；只有在电路运行的时候它的值才可能会改变。
线（Wire）可使用参数化的类型。

## 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# 常用操作符

现在您已经学习了模块`Module`的构造方式，我们接下来开始设计一些硬件！看看下面的空模块：

In [3]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
}

defined class MyModule

我们将这个模块称为`MyModule`，它从Chisel类型`Module`继承，这意味着它将用来生成Verilog中的硬件模块。`MyModule`模块有一个输入和一个输出，输入是一个4比特的无符号整数（`UInt`），输出也是同样的类型。

<span style="color:blue">**例子：Scala和Chisel的操作符看上去没有区别**</span><br>
我们可以对数据执行不同的操作：

In [4]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val two  = 1 + 1
  println(two)
  val utwo = 1.U + 1.U
  println(utwo)
  
  io.out := io.in
}
println(getVerilog(new MyModule))

[info] [0.002] Elaborating design...
2
UInt<1>(OpResult in cmd3$Helper$MyModule)
[info] [0.733] Done elaborating.
Total FIRRTL Compile Time: 493.4 ms
module cmd3HelperMyModule(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd3.sc 12:10]
endmodule



defined class MyModule

我们定义了两个`val`，第一个将两个Scala的`Int`类型的值相加，所以`println`打印出整数2；而第二个`val`是将两个*Chisel*的`UInt`类型相加，所以`println`将它看作硬件节点，并打印出类型名称和指针（`chisel3.core.UInt@d`-注意这里和notebook的结果不一致）。另外注意，`1.U`是一个类型转换，它从Scala的`Int`类型（1）转换成Chisel的`UInt`的literal值。

我们需要驱动输出，所以我们将它和输入相连接，就像上一节中的passthrough模块一样。

<span style="color:blue">**例子：不兼容的操作**</span><br>
如果将一个Chisel类型`1.U`和一个Scala类型`1`相加会发生什么？这两个类型是不兼容的，因为前者是描述硬件的线（Wire），它的值为1，而后者是Scala的值1，所以Chisel会给出类型不匹配的错误。

In [4]:
class MyModuleTwo extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val twotwo = 1.U + 1
  println(twotwo)
  
  io.out := io.in
}
println(getVerilog(new MyModule))

cmd4.sc:7: type mismatch;
 found   : Int(1)
 required: chisel3.core.UInt
  val twotwo = 1.U + 1
                     ^Compilation Failed

: 

在操作时区分类型非常重要，Scala是一种强类型语言，所以任何类型转换都必须是显式的。

<span style="color:blue">**例子：更多的Chisel操作**</span><br>
其他常见操作是减法和乘法。这些是对于无符号整数的操作，我们下面具体来看看这些操作。最后打印出了生成的Verilog（注意生成的Verilog代码里可能混有一些其他的功能）。

In [5]:
class MyOperators extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_add = Output(UInt(4.W))
    val out_sub = Output(UInt(4.W))
    val out_mul = Output(UInt(4.W))
  })

  io.out_add := 1.U + 4.U
  io.out_sub := 2.U - 1.U
  io.out_mul := 4.U * 2.U
}
println(getVerilog(new MyOperators))

[info] [0.001] Elaborating design...
[info] [0.012] Done elaborating.
Total FIRRTL Compile Time: 47.7 ms
module cmd4HelperMyOperators(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out_add,
  output [3:0] io_out_sub,
  output [3:0] io_out_mul
);
  wire [1:0] _T_3; // @[cmd4.sc 10:21]
  wire [4:0] _T_4; // @[cmd4.sc 11:21]
  assign _T_3 = 2'h2 - 2'h1; // @[cmd4.sc 10:21]
  assign _T_4 = 3'h4 * 3'h2; // @[cmd4.sc 11:21]
  assign io_out_add = 4'h5; // @[cmd4.sc 9:14]
  assign io_out_sub = {{2'd0}, _T_3}; // @[cmd4.sc 10:14]
  assign io_out_mul = _T_4[3:0]; // @[cmd4.sc 11:14]
endmodule



defined class MyOperators

下面是上面模块的测试代码。我们将显式地创建一个测试类，而不是像上一节中使用匿名测试类。这是编写测试的另一种方法。

In [6]:
class MyOperatorsTester(c: MyOperators) extends PeekPokeTester(c) {
  expect(c.io.out_add, 5)
  expect(c.io.out_sub, 1)
  expect(c.io.out_mul, 8)
}
assert(Driver(() => new MyOperators) {c => new MyOperatorsTester(c)})
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.005] Done elaborating.
Total FIRRTL Compile Time: 30.9 ms
Total FIRRTL Compile Time: 17.4 ms
End of dependency graph
Circuit state created
[info] [0.001] SEED 1561520754437
test cmd4HelperMyOperators Success: 3 tests passed in 5 cycles taking 0.019918 seconds
[info] [0.006] RAN 0 CYCLES PASSED
SUCCESS!!


defined class MyOperatorsTester

<span style="color:blue">**例子：多路复用（Mux)和连接(Concatenation)**</span><br>
除了加法，减法和乘法，Chisel还有多路复用（Mux)和连接(Concatenation)操作符。如下面代码所示。`Mux`操作类似于传统的三元操作符，其参数顺序为（select，如果select为true的值，如果select为false的值）。注意，`true.B`和`false.B`是创建Chisel Bool的首选方法。 
`Cat`操作符的参数顺序是高位，然后低位，它只有两个参数。连接两个以上的值需要多个`Cat`操作，或者在本教程的后面会介绍高级的Chisel和Scala功能。

In [7]:
class MyOperatorsTwo extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_mux = Output(UInt(4.W))
    val out_cat = Output(UInt(4.W))
  })

  val s = true.B
  io.out_mux := Mux(s, 3.U, 0.U) // 这里会返回 3.U，因为 s 的值是 true
  io.out_cat := Cat(2.U, 1.U)    // 连接 2 (b10) 和 1 (b1)，得到 5 (101)
}

println(getVerilog(new MyOperatorsTwo))
class MyOperatorsTwoTester(c: MyOperatorsTwo) extends PeekPokeTester(c) {
  expect(c.io.out_mux, 3)
  expect(c.io.out_cat, 5)
}
assert(Driver(() => new MyOperatorsTwo) {c => new MyOperatorsTwoTester(c)})
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.015] Done elaborating.
Total FIRRTL Compile Time: 33.6 ms
module cmd6HelperMyOperatorsTwo(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out_mux,
  output [3:0] io_out_cat
);
  assign io_out_mux = 4'h3; // @[cmd6.sc 9:14]
  assign io_out_cat = 4'h5; // @[cmd6.sc 10:14]
endmodule

[info] [0.000] Elaborating design...
[info] [0.006] Done elaborating.
Total FIRRTL Compile Time: 18.5 ms
Total FIRRTL Compile Time: 11.3 ms
End of dependency graph
Circuit state created
[info] [0.001] SEED 1561521203773
test cmd6HelperMyOperatorsTwo Success: 2 tests passed in 5 cycles taking 0.004585 seconds
[info] [0.004] RAN 0 CYCLES PASSED
SUCCESS!!


defined class MyOperatorsTwo
defined class MyOperatorsTwoTester

注意生成的Verilog中只包含常量，而不是真正生成了多路复用或连接的逻辑，这是因为FIRRTL的中间转换过程简化了电路，优化了逻辑。

关于完整的Chisel操作，见[Chisel cheatsheet](https://chisel.eecs.berkeley.edu/documentation.html)。最完整的操作以及它们的实现，参见[Chisel API](https://chisel.eecs.berkeley.edu/api/#package).

---
# 练习
完成这些练习可能需要参考[Chisel cheatsheet](https://chisel.eecs.berkeley.edu/documentation.html)。

<span style="color:red">**练习：MAC**</span><br>
创建一个Chisel模块，实现乘加-`(A*B)+C`，并通过测试。

In [ ]:
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  ???
}
class MACTester(c: MAC) extends PeekPokeTester(c) {
  val cycles = 100
  import scala.util.Random
  for (i <- 0 until cycles) {
    val in_a = Random.nextInt(16)
    val in_b = Random.nextInt(16)
    val in_c = Random.nextInt(16)
    poke(c.io.in_a, in_a)
    poke(c.io.in_b, in_b)
    poke(c.io.in_c, in_c)
    expect(c.io.out, in_a*in_b+in_c)
  }
}
assert(Driver(() => new MAC) {c => new MACTester(c)})
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>答案</strong></label>
<article>
<pre style="background-color:#f7f7f7">
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  io.out := (io.in_a * io.in_b) + io.in_c
}
</pre></article></div></section></div>

<span style="color:red">**练习：仲裁器（Arbiter）**</span><br>
下面的电路将来自FIFO的数据仲裁并分配给两个并行处理单元。 FIFO和并行处理单元（PE）之间通过ready-valid接口通信。这个仲裁器需要将数据发送到ready的那个PE，如果两个PE都ready的话，则优先发送给PE0。注意，当至少有一个PE可以接收数据时，仲裁器必须告诉FIFO它的状态是ready。另外，只有等PE ready之后，才能将数据置成valid。您很可能需要二元操作来完成这个练习。

<img src="images/arbiter.png" width="687" height="177">

In [ ]:
class Arbiter extends Module {
  val io = IO(new Bundle {
    // FIFO
    val fifo_valid = Input(Bool())
    val fifo_ready = Output(Bool())
    val fifo_data  = Input(UInt(16.W))
    
    // PE0
    val pe0_valid  = Output(Bool())
    val pe0_ready  = Input(Bool())
    val pe0_data   = Output(UInt(16.W))
    
    // PE1
    val pe1_valid  = Output(Bool())
    val pe1_ready  = Input(Bool())
    val pe1_data   = Output(UInt(16.W))
  })

  ???  
}
class ArbiterTester(c: Arbiter) extends PeekPokeTester(c) {
  import scala.util.Random
  val data = Random.nextInt(65536)
  poke(c.io.fifo_data, data)
  
  for (i <- 0 until 8) {
    poke(c.io.fifo_valid, (i>>0)%2)
    poke(c.io.pe0_ready,  (i>>1)%2)
    poke(c.io.pe1_ready,  (i>>2)%2)

    expect(c.io.fifo_ready, i>1)
    expect(c.io.pe0_valid,  i==3 || i==7)
    expect(c.io.pe1_valid,  i==5)
    
    if (i == 3 || i ==7) {
      expect(c.io.pe0_data, data)
    } else if (i == 5) {
      expect(c.io.pe1_data, data)
    }
  }
}
assert(Driver(() => new Arbiter) {c => new ArbiterTester(c)})
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>答案</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  io.fifo_ready := io.pe0_ready || io.pe1_ready
  io.pe0_valid := io.fifo_valid && io.pe0_ready
  io.pe1_valid := io.fifo_valid && io.pe1_ready && !io.pe0_ready
  io.pe0_data := io.fifo_data
  io.pe1_data := io.fifo_data
</pre></article></div></section></div>

<span style="color:red">**练习：参数化加法器（可选练习）**</span><br>
这个可选的练习目的是让您了解Chisel最强大的功能之一，它的参数化功能。为了暂时这个功能，我们将构造一个参数化的加法器，它可以在发生溢出时将结果饱和处理（saturate），或截断（truncate，wrap around）。

首先，看看下面的`Module`。我们将它的参数称为`saturate`，类型为*Scala*的`Boolean`。注意这不是Chisel的`Bool`类型，所以我们不会创建出一个即包含saturate又包含truncate的硬件加法器，而是创建一个*生成器*，它可以生成saturate的加法器*或者*truncate的加法器。这个决定是在编译时做出的。

接下来，注意输入和输出都是4比特的`UInt`类型。Chisel可以自动推断所需要的比特的宽度，如果您参考[cheatsheet](https://chisel.eecs.berkeley.edu/doc/chisel-cheatsheet3.pdf)的话，您可以看到普通加法的结果的宽度就等于两个输入中最宽的那个。这意味下面的代码：

```scala
val sum = io.in_a + io.in_b
```

会得到一个4比特宽的`sum`的线（wire），而它的值是被截断（truncate）的。如果需要检查加法的结果是否饱和的话，您需要一个5比特的线。这可以通过 `+&`操作得到。

```scala
val sum = io.in_a +& io.in_b
```

Finally, note that connecting a 4-bit `UInt` wire to a 5-bit `UInt` wire will truncate the MSB by default. You can use this to easily truncate the 5-bit sum for the non-saturating adder.
最后，注意，将4比特的`UInt`线连接到5比特的`UInt`的线上，默认会将高位截断。您可以用这一特性来轻松实现这个加法器中的截断功能。

In [ ]:
class ParameterizedAdder(saturate: Boolean) extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val out  = Output(UInt(4.W))
  })

  ???
}
class ParameterizedAdderTester(c: ParameterizedAdder, saturate: Boolean) extends PeekPokeTester(c) {
  // 100 random tests
  val cycles = 100
  import scala.util.Random
  import scala.math.min
  for (i <- 0 until cycles) {
    val in_a = Random.nextInt(16)
    val in_b = Random.nextInt(16)
    poke(c.io.in_a, in_a)
    poke(c.io.in_b, in_b)
    if (saturate) {
      expect(c.io.out, min(in_a+in_b, 15))
    } else {
      expect(c.io.out, (in_a+in_b)%16)
    }
  }
  
  // ensure we test saturation vs. truncation
  poke(c.io.in_a, 15)
  poke(c.io.in_b, 15)
  if (saturate) {
    expect(c.io.out, 15)
  } else {
    expect(c.io.out, 14)
  }
}
for (saturate <- Seq(true, false)) {
  assert(Driver(() => new ParameterizedAdder(saturate)) {c => new ParameterizedAdderTester(c, saturate)})
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>答案</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val sum = io.in_a +& io.in_b
  if (saturate) {
    io.out := Mux(sum > 15.U, 15.U, sum)
  } else {
    io.out := sum
  }
</pre></article></div></section></div>

---
# 本节结束!

[返回顶部](#top)